In [1]:
import node
import node.cuda

In [2]:
import numpy as np
import cupy as cp

In [3]:
class Classifier(node.Network):
    
    def __init__(self, num_in_ch):
        self.layers = [node.Conv2D(num_in_ch, 16, 3),
                       node.MaxPool2D(3, 2),
                       node.Conv2D(16, 32, 3),
                       node.MaxPool2D(3, 2),
                       node.Linear(512, 256),
                       node.Linear(256, 10)]
        
    def __call__(self, input):
        hidden = input
        hidden = self.layers[1](self.layers[0](hidden)).relu()
        hidden = self.layers[3](self.layers[2](hidden)).relu()
        hidden = hidden.reshape(input.value.shape[0], -1)
        hidden = self.layers[4](hidden).relu()
        hidden = self.layers[5](hidden)      
        return hidden
    
classifier = Classifier(1).gpu()
optimizer = node.Adam(classifier.get_parameters(), 0.001)

In [4]:
datasets = [node.MNIST(training=True), node.MNIST(training=False)]
data_loaders = [node.DataLoader(datasets[0], batch_size=100), node.DataLoader(datasets[1], batch_size=100)]

In [5]:
def train(input, target):
    optimizer.zero_grad()
    
    #　パラメーターを更新する
    output = classifier(input/255).softmax_with_cross_entropy(target)
    output.backward()
    optimizer()
    
    return output.cpu().value

In [6]:
def measure(prediction, target):
    # 出力とラベルを受け取り、何個正解したかの数を返す。
    prediction = cp.argmax(prediction.value, axis=1)
    target = cp.argmax(target.value, axis=1)
    
    return cp.asnumpy(cp.sum(cp.where(prediction == target, 1, 0)))

In [7]:
def evaluate(input, target):
    with node.zero_grad():
        prediction = classifier(input/255)
        output = prediction.softmax_with_cross_entropy(target)
        
    loss = output.cpu().value
    accuracy = measure(prediction, target)
    
    return loss, accuracy

In [8]:
for epoch in range(10):
    
    # Train Loss, Test Loss, Accuracy
    metrics = [0, 0, 0]

    for input, target in data_loaders[0]:
        metrics[0] += train(input.gpu(), target.gpu())
        
    for input, target in data_loaders[1]:
        loss, accuracy = evaluate(input.gpu(), target.gpu())
        metrics[1] += loss 
        metrics[2] += accuracy
        
    metrics[0] /= len(data_loaders[0])
    metrics[1] /= len(data_loaders[1])
    metrics[2] /= 100 * len(data_loaders[1])
    
    print("epoch {0:2}, training loss {1:.2f}, test loss {2:.2f}, accuracy {3:.2f}".format(epoch, *metrics))

epoch  0, training loss 0.54, test loss 0.21, accuracy 0.96
epoch  1, training loss 0.17, test loss 0.14, accuracy 0.98
epoch  2, training loss 0.13, test loss 0.19, accuracy 0.97
epoch  3, training loss 0.11, test loss 0.11, accuracy 0.98
epoch  4, training loss 0.08, test loss 0.10, accuracy 0.98
epoch  5, training loss 0.07, test loss 0.10, accuracy 0.99
epoch  6, training loss 0.06, test loss 0.10, accuracy 0.98
epoch  7, training loss 0.05, test loss 0.09, accuracy 0.99
epoch  8, training loss 0.04, test loss 0.09, accuracy 0.99
epoch  9, training loss 0.04, test loss 0.11, accuracy 0.98
